## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-13-14-53-26 +0000,startribune,Live: Pohlad family will not sell Minnesota Twins,https://www.startribune.com/live-pohlad-family...
1,2025-08-13-14-52-52 +0000,nyt,Trump’s Affinity for Putin Will Be Tested at H...,https://www.nytimes.com/2025/08/13/us/politics...
2,2025-08-13-14-51-37 +0000,bbc,"Claire's on brink of collapse putting 2,150 jo...",https://www.bbc.com/news/articles/cp8zwdy98k8o...
3,2025-08-13-14-46-00 +0000,wsj,Israel’s Military Chief Under Fire From Netany...,https://www.wsj.com/world/middle-east/israels-...
4,2025-08-13-14-43-53 +0000,nyt,Fatal U.S. Steel Plant Explosion Raises Questi...,https://www.nytimes.com/2025/08/12/us/us-steel...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2321/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,69
9,putin,19
3,will,18
88,ukraine,14
20,israel,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
97,2025-08-13-09-55-05 +0000,wapo,"Europe, Zelensky push to sway Trump before Ala...",https://www.washingtonpost.com/world/2025/08/1...,142
44,2025-08-13-12-47-52 +0000,nyt,Trump Will Discuss Ukraine With European Leade...,https://www.nytimes.com/2025/08/13/world/europ...,139
46,2025-08-13-12-37-12 +0000,nypost,Donald Trump Jr. jokes Mamdani will be ‘great’...,https://nypost.com/2025/08/13/us-news/don-trum...,135
93,2025-08-13-10-00-00 +0000,nypost,NY POSTcast Daily Debrief: Duffy on Buttigieg’...,https://nypost.com/2025/08/13/us-news/ny-postc...,134
35,2025-08-13-13-17-00 +0000,wsj,Trump Talks Ukraine Red Lines With Europe Befo...,https://www.wsj.com/world/trump-talks-ukraine-...,133


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
97,142,2025-08-13-09-55-05 +0000,wapo,"Europe, Zelensky push to sway Trump before Ala...",https://www.washingtonpost.com/world/2025/08/1...
46,66,2025-08-13-12-37-12 +0000,nypost,Donald Trump Jr. jokes Mamdani will be ‘great’...,https://nypost.com/2025/08/13/us-news/don-trum...
232,56,2025-08-12-19-21-00 +0000,wsj,The State Department released a long-delayed h...,https://www.wsj.com/politics/policy/state-depa...
261,53,2025-08-12-17-33-00 +0000,wsj,Crypto Entrepreneur Do Kwon Pleads Guilty to F...,https://www.wsj.com/finance/currencies/crypto-...
153,49,2025-08-13-01-37-14 +0000,nypost,"Left-wing dark-money megadonors, including Geo...",https://nypost.com/2025/08/12/us-news/dark-mon...
86,49,2025-08-13-10-00-00 +0000,nypost,Pete Buttigieg lowered standards for air traff...,https://nypost.com/2025/08/13/us-news/dot-chie...
152,47,2025-08-13-01-44-11 +0000,latimes,"She was killed while walking to her car, famil...",https://www.latimes.com/california/story/2025-...
235,40,2025-08-12-19-08-00 +0000,wsj,The White House plans to conduct a far-reachin...,https://www.wsj.com/politics/policy/white-hous...
151,40,2025-08-13-01-49-00 +0000,wsj,E.J. Antoni lacks the research record of previ...,https://www.wsj.com/politics/policy/ej-antoni-...
125,39,2025-08-13-05-46-11 +0000,bbc,Israel bombards Gaza City as UK and allies dem...,https://www.bbc.com/news/articles/clyj0dd0qj9o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
